In [206]:
import torch
import h5py
from torch import nn
from torchvision import transforms
from torch.utils import data
import numpy as np

In [68]:
transform = transforms.Compose([
            transforms.ToTensor()
        ])

In [89]:
def train_generator():
    with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(int(len(hf['X'])*0.8)): # both hf1 and hf2 have same length
                yield transform(hf['X'][i]), hf['y'][i]
                yield transform(hf2['X'][i]), hf2['y'][i]     

In [146]:
with h5py.File('data/electron.hdf5', 'r') as hf:
  test_ds_start=int(len(hf['X'])*0.8)+1
  len_ds=len(hf['X'])
test_ds_start

199201

In [91]:
def test_generator(): 
    with h5py.File('data/electron.hdf5','r') as hf2:
      with h5py.File('data/photon.hdf5', 'r') as hf:
          for i in range(int(len(hf['X'])*0.2)): # both hf1 and hf2 have same length
              yield transform(hf['X'][test_ds_start+i]), hf['y'][test_ds_start+i]
              yield transform(hf2['X'][test_ds_start+i]), hf2['y'][test_ds_start+i]  

In [156]:
class IterDataset(data.IterableDataset):
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        return self.generator()   

In [157]:
train_dataset = IterDataset(train_generator)
test_dataset = IterDataset(test_generator)
train_loader = data.DataLoader(train_dataset, batch_size=32)
test_loader = data.DataLoader(test_dataset, batch_size=32)

In [158]:
for i in train_loader:
    print(i.shape)
    break

AttributeError: 'list' object has no attribute 'shape'

In [196]:
model=nn.Sequential(
    nn.Conv2d(2, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    #nn.Conv2d(64, 12, kernel_size=3, stride=1, padding=1),
    #nn.BatchNorm2d(64),
    #nn.ReLU(),
    #nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(4096, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
).to('cuda')

In [197]:
from torchsummary import summary

summary(model, (2, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             608
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          18,496
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
           Flatten-9                 [-1, 4096]               0
           Linear-10                  [-1, 256]       1,048,832
             ReLU-11                  [-1, 256]               0
           Linear-12                    [-1, 1]             257
          Sigmoid-13                    [-1, 1]               0
Total params: 1,068,385
Trainable param

In [198]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [199]:
from sklearn.metrics import roc_auc_score

In [200]:
batch_size=32

In [ ]:
for epoch in range(3):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images.to('cuda'))
        labels = labels.type(torch.FloatTensor)
        loss = criterion(outputs, labels.to('cuda').unsqueeze(1))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            y_pred = np.where(outputs.cpu().data.numpy().flatten() > 0.5, 1, 0)
            print(outputs.cpu().data.min())
            y_true = labels.cpu().data.numpy()
            roc_auc = roc_auc_score(y_true, y_pred) 
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, ROC:{} ' 
                   .format(epoch+1, 10, i+1,  test_ds_start//batch_size, loss.item(), roc_auc))

tensor(0.2348)
Epoch [1/10], Step [100/6225], Loss: 0.6691, ROC:0.6875 
tensor(0.2339)
Epoch [1/10], Step [200/6225], Loss: 0.6336, ROC:0.65625 
tensor(0.2547)
Epoch [1/10], Step [300/6225], Loss: 0.6850, ROC:0.59375 
tensor(0.2219)
Epoch [1/10], Step [400/6225], Loss: 0.6552, ROC:0.59375 
tensor(0.3242)
Epoch [1/10], Step [500/6225], Loss: 0.7218, ROC:0.5 
tensor(0.1308)
Epoch [1/10], Step [600/6225], Loss: 0.5961, ROC:0.65625 
tensor(0.2440)
Epoch [1/10], Step [700/6225], Loss: 0.6222, ROC:0.65625 
tensor(0.1174)
Epoch [1/10], Step [800/6225], Loss: 0.6294, ROC:0.625 
tensor(0.1995)
Epoch [1/10], Step [900/6225], Loss: 0.7413, ROC:0.5625 
tensor(0.1521)
Epoch [1/10], Step [1000/6225], Loss: 0.5792, ROC:0.6875 
tensor(0.1364)
Epoch [1/10], Step [1100/6225], Loss: 0.6383, ROC:0.625 
tensor(0.1812)
Epoch [1/10], Step [1200/6225], Loss: 0.6940, ROC:0.5 
tensor(0.2489)
Epoch [1/10], Step [1300/6225], Loss: 0.6452, ROC:0.59375 
tensor(0.1126)
Epoch [1/10], Step [1400/6225], Loss: 0.6535, R

tensor(0.1262)
Epoch [1/10], Step [11400/6225], Loss: 0.6705, ROC:0.65625 
tensor(0.1116)
Epoch [1/10], Step [11500/6225], Loss: 0.6135, ROC:0.65625 


In [ ]:
# test model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor).to('cuda')
        outputs = model(images.to('cuda'))
        _, predicted = torch.max(outputs.data, 1)
        y_pred = outputs.data.max(1, keepdim=True)[1]
        y_true = labels.data
        roc_auc = roc_auc_score(y_true, y_pred) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
